In [10]:
# Importing the necessary libraries
from importlib import reload
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import metrics
import math

import utils
import data

reload(utils)

<module 'utils' from '/root/team-skin-lesions/utils.py'>

In [11]:
label_names = ['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']
NUM_OF_CLASSES = len(label_names)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))



def build_model(n_classes, learning_rate=0.01, loss="categorical_crossentropy"):
    """Build a multi-class logistic regression model using Keras.

      Args:
        n_classes: Number of classes in the dataset
        learning_rate: The desired learning rate for SGD.

      Returns:
        model: A tf.keras model (graph).
    """
    tf.keras.backend.clear_session()
    np.random.seed(0)
    tf.random.set_seed(0)

    model = keras.Sequential()
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(
        units=n_classes,
        activation='softmax',
        use_bias=True, 
        kernel_initializer=tf.ones_initializer
    ))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(loss=loss, optimizer=optimizer, metrics=[tf.keras.metrics.Recall(),tf.keras.metrics.Precision(), tf.keras.metrics.Accuracy()])
    
    return model

In [17]:
utils.select_subset_for_playground(100)

batch_size = 1
img_height = 244
img_width = 244
random_seed=123

train_ds = tf.keras.utils.image_dataset_from_directory(
    'isic_data/playground/',  # switch to 'training' when ready
    class_names=sorted(data.SkinClass.__members__.keys()),
    validation_split=0.2,
    subset="training",  # switch to 'validation' for the 20%
#     seed=random_seed,
    image_size=(img_height, img_width),  # default 256x256
    batch_size=batch_size,
    shuffle=False,
    label_mode='categorical',
)

for layer in [tf.keras.layers.Rescaling(1./255),]:
    train_ds = train_ds.map(lambda x, y: (layer(x), y))

model = build_model(NUM_OF_CLASSES)

history = model.fit(train_ds, epochs=100, batch_size=10, verbose=0)

y_pred = model.predict(train_ds)

history = pd.DataFrame(history.history)
display(history)
                                      
#print(y_pred)

FileNotFoundError: [Errno 2] No such file or directory: 'isic_data/undersampled/MEL'

In [18]:
batch_size = 1
img_height = 244
img_width = 244
random_seed=123

validation_ds = tf.keras.utils.image_dataset_from_directory(
    'isic_data/playground/',  # switch to 'training' when ready
    class_names=sorted(data.SkinClass.__members__.keys()),
    validation_split=0.2,
    subset="validation",  # switch to 'validation' for the 20%
#     seed=random_seed,
    image_size=(img_height, img_width),  # default 256x256
    batch_size=batch_size,
    shuffle=False,
    label_mode='categorical',
)

for layer in [tf.keras.layers.Rescaling(1./255),]:
    validation_ds = validation_ds.map(lambda x, y: (layer(x), y))

y_pred = model.predict(validation_ds)
#print(y_pred)
y_true = list()

for i in range(len(y_pred)): 
    y_true.append(list(validation_ds.as_numpy_iterator())[i][1][0]) 
#print(y_true)

m = tf.keras.metrics.Precision()
m.update_state(y_true, y_pred)
print("Precision: " +  str(m.result().numpy()))

recall = tf.keras.metrics.Recall()
recall.update_state(y_true, y_pred)
print("Recall: " +  str(recall.result().numpy()))

accuracy = tf.keras.metrics.Accuracy()
accuracy.update_state(y_true, y_pred)
print("Accuracy: " +  str(accuracy.result().numpy()))



ValueError: The `class_names` passed did not match the names of the subdirectories of the target directory. Expected: ['images'], but received: ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']

In [ ]:
#train_ds.class_names

In [ ]:
img_list = list(train_ds.as_numpy_iterator())[0][0][0]
print(img_list)
utils.plot_image()

In [ ]:
list(train_ds.as_numpy_iterator())[0][1][0]


In [ ]:
print(y_pred)

In [ ]:
# Print True Y 
ll = list(train_ds.as_numpy_iterator())
for i in range(len(ll)): 
    print(str(i) + " " + str(ll[i][1]))